In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib as plt
import h5py
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [ ]:
os.getcwd()
os.chdir('/home/vividlee/Desktop/YeloCar/HDF5/deep4pc_test5/')

hdf5_files = os.listdir()
print(hdf5_files)

In [ ]:
os.chdir('/home/vividlee/Desktop/YeloCar')
os.getcwd()

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory ' +  directory)

dataset_version = 'deep4pc_test5'
model_version = 'model_' + '1_p5'

# createFolder('./Test/' + model_version)

with h5py.File('./Model/Model_test5_existing-data+KHG-data/dataset_hsv_test5_existing-data+KHG-data.hdf5', 'r') as hf:
    image_train = np.array(hf['x_train'][:, :, :, :])
    action_train = np.array(hf['y_train'][:, :])
    image_val = np.array(hf['x_val'][:, :, :, :])
    action_val = np.array(hf['y_val'][:, :])
    
    
(x_train, y_train), (x_test, y_test) = (image_train, action_train), (image_val, action_val)

x_train /= 255.
x_test /= 255.

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(48, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

# optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mae', optimizer='adam', metrics=['mae', 'mse'])

model.summary()

model_path = './Model/Model_test5_existing-data+KHG-data' + '/' + model_version + '_{epoch:02d}_{loss:.4f}_{val_mae:.4f}'

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_mae', verbose=1, save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=5)

history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping_callback, checkpointer])

print("\n")
print(model.evaluate(x_test, y_test))

In [ ]:
y_vloss = history.history['val_loss']   # testset
y_loss = history.history['loss']        # trainset
print(y_vloss)
print('\n')
print(y_loss)
print('\n')
y_vmae = history.history['val_mae']
y_mae = history.history['mae']
print(y_vmae)
print('\n')
print(y_mae)
print('\n')
x_len = np.arange(len(y_loss))
print(len(y_loss))
print('\n')

plt.figure(figsize=(10, 8))
plt.plot(x_len+1, y_vloss, 'ro--', label='val_loss')
plt.plot(x_len+1, y_loss, 'bo-', label='loss')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.xlim([0.5, len(y_loss)+0.5])
plt.ylim([0.03, 0.08])
plt.xticks([1, 3, 5, 7, 9, 11, 13, 15],fontsize=13)
plt.yticks(fontsize=13)
plt.grid()
plt.show()

plt.figure(figsize=(10, 8))
plt.plot(x_len+1, y_vmae, 'ro--', label='val_mae')
plt.plot(x_len+1, y_mae, 'bo-', label='mae')
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('MAE', fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.xlim([0.5, len(y_loss)+0.5])
plt.ylim([0.03, 0.08])
plt.xticks([1, 3, 5, 7, 9, 11, 13, 15], fontsize=13)
plt.yticks(fontsize=13)
plt.grid()
plt.show